In [8]:
pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import os
import cv2
import numpy as np
from keras.utils import normalize
from matplotlib import pyplot as plt

In [28]:
patch_size=256


In [35]:

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters): # Duas convoluções 
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)   # ReLu como função de ativação

    return x


#Encoder block: Conv block followed by maxpooling
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters) # gerar features
    p = MaxPool2D((2, 2))(x)    # Reduz as imagens a aprtir de um kernel 2x2
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])   # utiliza a posição dos pixels do encoder
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    #Change the activation based on n_classes
    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:               # mulsti-class classification
      activation = 'softmax'

    # Output convolution
    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [36]:
def get_model():
    return build_unet(input_shape, n_classes)

input_shape = (patch_size, patch_size, 1)
n_classes = 6

In [37]:
def prediction(model, image, patch_size):
    segm_img = np.zeros(image.shape[:2])  #Array with zeros to be filled with segmented values
    patch_num=1
    for i in range(0, image.shape[0], 256):   #Steps of 256
        for j in range(0, image.shape[1], 256):  #Steps of 256
            #print(i, j)
            single_patch = image[i:i+patch_size, j:j+patch_size]
            single_patch_norm = np.expand_dims(normalize(np.array(single_patch), axis=1),2)
            single_patch_shape = single_patch_norm.shape[:2]
            single_patch_input = np.expand_dims(single_patch_norm, 0)
            single_patch_prediction = (model.predict(single_patch_input)[0,:,:,0] > 0.5).astype(np.uint8)
            segm_img[i:i+single_patch_shape[0], j:j+single_patch_shape[1]] += cv2.resize(single_patch_prediction, single_patch_shape[::-1])
          
            print("Finished processing patch number ", patch_num, " at position ", i,j)
            patch_num+=1
    return segm_img

In [38]:
##########
#Load model and predict
model = get_model()
#model.load_weights('mitochondria_gpu_tf1.4.hdf5')
model.load_weights('/content/drive/MyDrive/MEV - metamaficas/Projeto_Segmentação_Rocha_Máfica/Modelos/GIMP_348img_b12_e100.hdf5')


softmax


In [40]:
path = '/content/drive/MyDrive/MEV - metamaficas/Classif_AT-Z-2'


for image in os.listdir(path):
    #Large image
    large_image = cv2.imread(path+image, 0)
    segmented_image = prediction(model, large_image, patch_size)
    plt.hist(segmented_image.flatten())  #Threshold everything above 0

    plt.imsave(f'/content/drive/MyDrive/MEV - metamaficas/Masks_AT-Z-2/{image}.tif', segmented_image, cmap='gray')

    plt.figure(figsize=(8, 8))
    plt.subplot(221)
    plt.title('Large Image')
    plt.imshow(large_image, cmap='gray')
    plt.subplot(222)
    plt.title('Prediction of large Image')
    plt.imshow(segmented_image, cmap='gray')
    plt.show()


AttributeError: ignored